---

# 电影推荐实例--基于协同过滤和DL特征提取的比较

> [电影推荐实例--基于协同过滤和DL特征提取的比较](https://blog.csdn.net/qq_32453673/article/details/72593675)

## 1. 数据集

来源于 [MovieLens](https://grouplens.org/datasets/movielens/) 中的 ml-latest-small.zip，当然也可以从本文最后我的[github](https://github.com/elliotzhao/Recommend-movies) 中找到。 

本项目主要用到其中的两个csv文件：

- ml-latest-small/ratings.csvz：包含 671 个用户，共 100004 个打分

- ml-latest-small/links.csv：包含 9125 个电影及其 IMDBid 和 TMDBid

分别长这样：

```shell
#ratings.csvz：
   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205
......

#links.csv：
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0
......
```

## 2. 理论背景

粗略地说，推荐系统有三种类型（不包括简单的评级方法）：

- 基于内容的推荐

- 协同过滤

- 混合模型

“基于内容的推荐”是一个回归问题，我们把电影内容作为特征，对用户对电影的评分做预测。

“协同过滤”中，一般无法提前获得内容特征。是通过用户之间的相似度（用户们给了用一个电影相同的评级）和电影之间的相似度（有相似用户评级的电影）来学习潜在特征，同时预测用户对电影的评分。在学习了电影的特征之后，我们便可以衡量电影之间的相似度，并根据用户历史观影信息，向他/她推荐最相似的电影。

“基于内容的推荐”和“协同过滤”是10多年前最先进的技术。很显然，现在有很多模型和算法可以提高预测效果。比如，针对事先缺乏用户电影评分信息的情况，可以使用隐式矩阵分解，用偏好和置信度取代用户电影打分——比如用户对电影推荐有多少次点击，以此进行协同过滤。另外，我们还可以将“内容推荐”与“协同过滤”的方法结合起来，将内容作为侧面信息来提高预测精度。这种混合方法，可以用“学习进行排序”（”Learning to Rank” ）算法来实现。

在该项目中，采用的方法是“协同过滤”。首先，用电影和用户相似度来找出相似度最高的海报，并基于相似度做电影推荐。然后，我将讨论如何Deep Learning学习潜在特征、做电影推荐。最后会谈谈如何在推荐系统中使用深度学习。

## 3. 电影相似性

对于基于协同过滤的推荐系统，首先要建立评分矩阵。其中，每一行表示一个用户，每一列对应其对某一电影的打分。建立的评分矩阵如下：


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

rating_f = 'ml-latest-small/ratings.csv'
link_f   = 'ml-latest-small/links.csv'

df    = pd.read_csv(rating_f, sep=',')
df_id = pd.read_csv(link_f, sep=',')
df    = pd.merge(df, df_id, on=['movieId'])

rating_matrix = np.zeros((df.userId.unique().shape[0], max(df.movieId)))
for row in df.itertuples():
    rating_matrix[row[1] - 1, row[2] - 1] = row[3]
rating_matrix = rating_matrix[:,:9000] #get first 9000 movies

“ratings.csv”包含用户id，电影id, 评级，和时间信息，其中同一个用户id可能对多个电影进行打分；

“links.csv”包含电影id, IMDB id,和TMDB id。

这两个文件中包含的电影总数是不同的（rating.csv: 9066, links.csv: 9125）。我们将两个文件依movieId合并，并在此基础上得到评分矩阵rating_matrix[userId:movieId]=grade。

计算评分矩阵的稀疏性：

In [2]:
#Evaluate sparsity of matrix
sparsity = float(len(rating_matrix.nonzero()[0]))
sparsity /= (rating_matrix.shape[0] * rating_matrix.shape[1])
sparsity *= 100
print('Sparsity is {0}%'.format(sparsity))

Sparsity is 1.4030468620632555%


可以得到sparsity=1.4%，认为是稀疏矩阵。 

现在，为了训练和测试，我们将评分矩阵分解成两个矩阵，从每行（userId）评分矩阵中抠出了10个评分，将其放入测试集。

In [3]:
#Splite to train/test matrix
train_matrix = rating_matrix.copy()
test_matrix  = np.zeros(rating_matrix.shape)

for i in range(rating_matrix.shape[0]):
        rating_index = np.random.choice(rating_matrix[i].nonzero()[0], size=10, replace=True) #return a list
        train_matrix[i, rating_index] = 0.0
        test_matrix[i, rating_index] = rating_matrix[i, rating_index]

根据余弦相似性计算两个特征间的夹角（具体过程参见另外一篇博文：[余弦相似定理和新闻分类](https://blog.csdn.net/qq_32453673/article/details/72615773)）： 

$$cos(\theta )=\frac{<b,c>}{\mid b\mid\cdot \mid c\mid}$$

即：

$$cos(\theta )=\frac{x_{1}y_{1}+x_{2}y_{2}+\cdot \cdot \cdot +x_{n}y_{n} }{\sqrt{x_{1}^{2}+x_{2}^{2}+\cdot \cdot \cdot +x_{n}^{2}}\cdot \sqrt{y_{1}^{2}+y_{2}^{2}+\cdot \cdot \cdot +y_{n}^{2}}}$$

同理计算用户/电影中的余弦相似性： 

$$s(u,v)=\frac{r_{u}r_{v}}{\parallel r_{u}\parallel \parallel r_{v}\parallel }$$

这里 s(u,v) 是用户 u 和 v 之间的余弦相似度。


In [4]:
#Cosine similarity
similarity_usr = train_matrix.dot(train_matrix.T) + 1e-9
norms = np.array([np.sqrt(np.diagonal(similarity_usr))])
similarity_usr = (similarity_usr / (norms * norms.T))

similarity_mv = train_matrix.T.dot(train_matrix) + 1e-9
norms = np.array([np.sqrt(np.diagonal(similarity_mv))])
similarity_mv = (similarity_mv / (norms * norms.T))

我们利用其他所有用户对某一电影i的评分来预测当前用户u对该电影的评分，并且用相似度作为权重，然后标准化： 

$$\hat{r}_{uv}=\frac{\sum_{v} s(u,v)r_{vi}}{\sum_{v}\mid s(u,v)\mid }$$

In [5]:
prediction = similarity_usr.dot(train_matrix) / np.array([np.abs(similarity_usr).sum(axis=1)]).T
prediction = prediction[test_matrix.nonzero()]
test_vector = test_matrix[test_matrix.nonzero()]
mse = mean_squared_error(prediction, test_vector)

print('mse: {0}'.format(mse))

mse: 9.864265738978938


预测的MSE是9.8。到此我们实现了根据其他用户的评分来预测某一个用户对某个电影的评分并且进行评价。

但这个数根本看不出来是什么鬼，也不知道预测的怎么样。因此我们换一个直观的方法，直接看相似度：由于矩阵similarity_mv[i,j]表示第i个电影和第j个电影的相关性，所以对于第i个电影找出该行最大的5个值所在的列号，就是和这个电影最相似的电影的movieId。 
我们使用IMDB id，使用API从Movie Database 网站获取海报。

从“links.csv”获得movieId和IMDB id的映射关系：

In [6]:
n_display   = 5
base_mv_idx = 0
idx_to_mv   = {}

for row in df_id.itertuples():
        idx_to_mv[row[1]-1] = row[2]

base_mv_idx表示要找出和这个movieId相似的电影，n_display表示找这么多个相似的。

In [7]:
mv = [idx_to_mv[x] for x in np.argsort(similarity_mv[base_mv_idx])[:-n_display-1:-1]]
mv = filter(lambda imdb: len(str(imdb))==6, mv)
mv = list(mv)[:n_display]

下面要把这n_display个电影的海报通过API从Movie Database上扒下来，先获得存储它们的URL：

In [8]:
import requests
import json

#Get posters from Movie Database by API
headers = {'Accept':'application/json'}
payload = {'api_key':'20047cd838219fb54d1f8fc32c45cda4'}
response = requests.get('http://api.themoviedb.org/3/configuration',params=payload,headers=headers)
response = json.loads(response.text)

base_url = response['images']['base_url']+'w185'

def get_poster(imdb,base_url):
    #query themovie.org API for movie poster path.
    imdb_id = 'tt0{0}'.format(imdb)
    movie_url = 'http://api.themoviedb.org/3/movie/{:}/images'.format(imdb_id)
    response = requests.get(movie_url,params=payload,headers=headers)
    try:
        file_path = json.loads(response.text)['posters'][0]['file_path']
    except:
        print('Something wrong, cannot get the poster for imdb id: {0}!'.format(imdb))

    return base_url+file_path

In [9]:
URL = [0] * len(list(mv))

for i, m in enumerate(mv):
    URL[i] = get_poster(m, base_url)

根据得到的URL将这n_display个电影的海报一起显示出来：

In [10]:
from IPython.display import Image
from IPython.display import display
from IPython.display import HTML

images = ''
for i in range(len(mv)):
    print(URL[i])
    images+="<img style='width: 100px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % URL[i]

    #print(images)
display(HTML(images))

http://image.tmdb.org/t/p/w185/uMZqKhT4YA6mqo2yczoznv7IDmv.jpg
http://image.tmdb.org/t/p/w185/kuTPkbQmHxBHsxaKMUL1kUchhdE.jpg
http://image.tmdb.org/t/p/w185/z4ROnCrL77ZMzT0MsNXY5j25wS2.jpg
http://image.tmdb.org/t/p/w185/bqLlWZJdhrS0knfEJRkquW7L8z2.jpg


In [11]:
## 完整代码
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

rating_f = 'ml-latest-small/ratings.csv'
link_f   = 'ml-latest-small/links.csv'

df = pd.read_csv(rating_f,sep=',')
df_id = pd.read_csv(link_f,sep=',')
df = pd.merge(df,df_id,on=['movieId'])

rating_matrix = np.zeros((df.userId.unique().shape[0],max(df.movieId)))
for row in df.itertuples():
    rating_matrix[row[1]-1,row[2]-1] = row[3]
rating_matrix = rating_matrix[:,:9000]

#Evaluate sparsity of matrix
sparsity = float(len(rating_matrix.nonzero()[0]))
sparsity /= (rating_matrix.shape[0]*rating_matrix.shape[1])
sparsity *= 100
print('Sparsity is {0}%'.format(sparsity))

#Splite to train/test matrix
train_matrix = rating_matrix.copy()
test_matrix = np.zeros(rating_matrix.shape)

for i in range(rating_matrix.shape[0]):
    rating_index = np.random.choice(rating_matrix[i,:].nonzero()[0],size=10,replace=True)
    train_matrix[i,rating_index] = 0.0
    test_matrix[i,rating_index] = rating_matrix[i,rating_index]

#Cosine similarity
similarity_usr = train_matrix.dot(train_matrix.T) + 1e-9
norms = np.array([np.sqrt(np.diagonal(similarity_usr))])
similarity_usr = (similarity_usr/(norms*norms.T))

similarity_mv = train_matrix.T.dot(train_matrix) + 1e-9
norms = np.array([np.sqrt(np.diagonal(similarity_mv))])
similarity_mv = (similarity_mv/(norms*norms.T))

prediction = similarity_usr.dot(train_matrix)/np.array([np.abs(similarity_usr).sum(axis=1)]).T
prediction = prediction[test_matrix.nonzero()]
test_vector = test_matrix[test_matrix.nonzero()]
mse = mean_squared_error(prediction,test_vector)

print('mse: {0}'.format(mse))

#Test
import requests
import json
from IPython.display import Image
from IPython.display import display
from IPython.display import HTML

k = 10
n_display = 5
base_mv_idx = 0
idx_to_mv = {}

for row in df_id.itertuples():
    idx_to_mv[row[1]-1] = row[2]
mv = [idx_to_mv[x] for x in np.argsort(similarity_mv[base_mv_idx])[:-k-1:-1]]
mv = filter(lambda imdb: len(str(imdb))==6, mv)
mv = list(mv)[:n_display]

#Get posters from Movie Database by API
headers = {'Accept':'application/json'}
payload = {'api_key':'20047cd838219fb54d1f8fc32c45cda4'}
response = requests.get('http://api.themoviedb.org/3/configuration',params=payload,headers=headers)
response = json.loads(response.text)

base_url = response['images']['base_url']+'w185'

def get_poster(imdb,base_url):
    #query themovie.org API for movie poster path.
    imdb_id = 'tt0{0}'.format(imdb)
    movie_url = 'http://api.themoviedb.org/3/movie/{:}/images'.format(imdb_id)
    response = requests.get(movie_url, params=payload, headers=headers)

    try:
        file_path = json.loads(response.text)['posters'][0]['file_path']
    except:
        print('Something wrong, cannot get the poster for imdb id: {0}!'.format(imdb))

    return base_url+file_path


URL = [0]*len(mv)
for i,m in enumerate(mv):
    URL[i] = get_poster(m,base_url)

images = ''
for i in range(len(mv)):
    images+="<img style='width: 100px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % URL[i]

#print(images)
display(HTML(images))

Sparsity is 1.4030468620632555%
mse: 9.657992950879118
